In [1]:
import pandas as pd
import numpy as np
from ECIRModel import ECIRModel
from CIRModel import BasicCIRModel
from yieldCurvePrediction import yieldCurvePredictor

In [23]:
class strategy:

    def __init__(self, model, dates, data):

        self.model = model
        self.dates = dates
        self.data = data

    def prices(self):

        mod_dict = {}
        price_dict = {}
        
        for date in self.dates:
    
            mod_dict[date] = yieldCurvePredictor(self.model, date, self.data) 
            price_dict[date] = mod_dict[date].comb_data()[1]
    
        return price_dict

    def buy_sell(self):

        prices = self.prices()
    
        for date in self.dates:
    
            bonds = []
        
            for bond in prices[date].index:
                if prices[date]['Simulated Prices'][bond] < prices[date]['Actual Prices'][bond]:
                    bonds.append(1)
                else:
                   bonds.append(-1)
            prices[date]['Buy?'] = bonds 

        return prices

    def profit(self, cash = 1000000):

        prices = self.buy_sell()
        weights = 1 / len(prices[self.dates[0]])
        value = weights*cash
        invested = np.zeros(len(prices[self.dates[0]]))
        shares = np.zeros(len(prices[self.dates[0]]))
        profit = np.zeros(len(prices[self.dates[0]]))
        prices = self.buy_sell()
        
        for i in range(len(self.dates)):

            errs = pd.DataFrame({'Error' :np.abs(prices[self.dates[i]]['Actual Prices'] - prices[self.dates[i]]['Simulated Prices'])})

            errs = errs.sort_values(by = 'Error')
            errs['Invest?'] = np.zeros(len(errs))
            new_index = errs.index
            index_ones = new_index[:5]
            index_zeros = new_index[5:]
            errs['Invest?'][index_ones] = np.ones(5)
            errs['Invest?'][index_zeros] = np.zeros(len(errs) - 5)

            prices[dates[i]] = prices[dates[i]].join(errs)
            
            for j in range(len(prices[self.dates[i]]['Actual Prices'])):
                
                profit[j] += (shares[j] * prices[self.dates[i]]['Actual Prices'][j]) - invested[j]
                yesterday_invested = np.sum(invested)
                
                shares[j] = np.floor(value / prices[self.dates[i]]['Actual Prices'][j]) * prices[self.dates[i]]['Invest?'][j]
                invested[j] = (shares[j] * prices[self.dates[i]]['Actual Prices'][j]) * prices[self.dates[i]]['Invest?'][j]
                
                if prices[self.dates[i]]['Buy?'][j] == -1:
                    invested[j] *= -1
                    shares[j] *= -1
        
            cash += (np.sum(profit) + yesterday_invested - np.sum(invested))
            value = weights*cash

            print(np.sum(profit))

        return pd.DataFrame({'Profit' : profit}).set_index(prices[dates[0]].index)

In [24]:
# Load the DGS_30 data
dgs_30_data = pd.read_csv('Data Folder/DGS_30.csv')
# Load the DGS3MO data
dgs3mo_data = pd.read_csv('Data Folder/DGS3MO.csv')

# Convert the 'DATE' column to datetime and set it as index for both DataFrames
dgs_30_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
dgs_30_data['Date'] = pd.to_datetime(dgs_30_data['Date'])
dgs_30_data.set_index('Date', inplace=True)

dgs3mo_data.rename(columns={'DATE': 'Date'}, inplace=True)
dgs3mo_data['Date'] = pd.to_datetime(dgs3mo_data['Date'])
dgs3mo_data.set_index('Date', inplace=True)

# Insert the DGS3MO data into the DGS_30 data as the first column
dgs_30_data.insert(0, 'DGS3MO', dgs3mo_data['DGS3MO'])

data=dgs_30_data

dates = data.loc[data.index >= '2024-03-19'].index

params_ecir = {
    "kappa": 0.01227,    
    "mu_r": 0.09854,     
    "sigma": 0.09395,    
    "mu": 0.002995,      
    "gamma": 0.002321,   
    "r": 66,             
    "p": 0.3049          
}

params_cir = {
    "kappa": 1.2310,
    "mu_r": 0.0459,
    "sigma": 0.1410
}

In [25]:
CIR = strategy(BasicCIRModel(**params_cir), dates, data)
ECIR = strategy(ECIRModel(**params_ecir), dates, data)
        
CIR_profit = CIR.profit(1000000).rename(columns = {'Profit' : 'CIR'})
ECIR_profit = ECIR.profit(1000000).rename(columns = {'Profit' : 'ECIR'})

profits = pd.concat([CIR_profit, ECIR_profit], axis = 1)

0.0
13.543832927356561
13.459472036720399
-61.71358548124044
-70.37600763301089
-97.1777284716809
-107.32306848977532
-77.40621863995693
-38.72501025637757
58.59327580741592
89.79388183906485
83.63571954048166
58.740127159173426
34.1741616547406
38.541088202218816
-3.9315492852329044
-183.7540630746771
-171.06860849214718
-208.98961381229674
-364.2168792394623
0.0
185.77379542264316
152.1841440337812
332.1217936703033
269.67085962256533
286.1124906199366
391.87388988485327
274.9260194294984
196.72309415606287
-77.88921646172457
-75.9591683196777
-5.091592745648086
70.73568255827195
-155.89816618182886
-311.0171769101107
-108.98513642271791
-858.2057766157268
-831.6830266532306
-641.7581015357355
-947.9404765168474


In [27]:
np.sum(profits)

CIR    -364.216879
ECIR   -947.940477
dtype: float64